In [2]:
import os
from typing import List, Dict, Any, Optional
import asyncio
import warnings
import json

import dotenv
import pandas as pd
from enum import Enum
from pydantic import BaseModel, Field
from pydantic_ai import Agent, BinaryContent, ModelRetry
from pydantic_ai.messages import ModelRequest, ModelResponse
from pydantic_ai.models.bedrock import BedrockConverseModel, BedrockModelSettings

warnings.filterwarnings("ignore", category=UserWarning)

# Load environment variables from .env file
dotenv_path = os.path.join('..', '.env')
dotenv.load_dotenv(dotenv_path)


True

In [3]:
def generate_agent(
    model_id: str,
    output_type: Optional[BaseModel | str] = str,
    retries: int = 5,
    temperature: float = 0.2,
    max_tokens: int = 20000,
    timeout: int = 60,
) -> Agent:

    model = BedrockConverseModel(model_name=model_id)

    model_settings = BedrockModelSettings(
        temperature=temperature,
        max_tokens=max_tokens,
        timeout=timeout,
    )

    agent = Agent(
        model,
        retries=retries,
        output_type=output_type,
        model_settings=model_settings,
        system_prompt = """
        あなたは講義に関するフィードバックを分析するAIアシスタントです。
        以下のルールに従って、コメントを分類してください。
        1. コメントの感情を分類してください。
        2. コメントのカテゴリを分類してください。
        3. コメントの重要度を分類してください。
        4. コメントの共通性を分類してください。
        
        x`各分類は以下の選択肢から選んでください。
        
        感情: ポジティブ, 中立, ネガティブ
        カテゴリ: 講義内容, 講義資料, 運営, その他
        重要度: 高, 中, 低
        共通性: 高, 中, 低
        """
    )

    return agent


async def generate_content_with_agent(
    model_id: str,
    query: list[str | BinaryContent],
    history: list[ModelRequest | ModelResponse] = [],
    output_type: Optional[BaseModel | str] = str,
    retries: int = 3,
    temperature: float = 0.2,
    max_tokens: int = 20000,
    timeout: int = 60,
) -> tuple[str, int, int, int]:

    await asyncio.sleep(0.5)  # APIレート制限対策

    agent = generate_agent(
        model_id,
        output_type,
        retries=retries,
        temperature=temperature,
        max_tokens=max_tokens,
        timeout=timeout,
    )

    response = await agent.run(
        query,
        message_history=history,
    )
    output = response.output

    input_tokens = response.usage().request_tokens
    output_tokens = response.usage().response_tokens
    total_tokens = response.usage().total_tokens
    think_tokens = total_tokens - input_tokens - output_tokens


    return output, input_tokens, output_tokens, think_tokens

def calculate_cost(
    model_name: str,
    input_tokens: int,
    output_tokens: int,
    think_tokens: Optional[int] = None,
) -> tuple[float, float, float]:
    """
    Calculate the total cost based on input, output, and think tokens.
    """
    if think_tokens is None:
        think_tokens = 0

    if "gemini-2.0-flash" in model_name:
        input_cost_per_mil_token = 0.15
        output_cost_per_mil_token = 0.60

    elif "gemini-2.5-flash" in model_name:
        input_cost_per_mil_token = 0.15
        if think_tokens == 0:
            output_cost_per_mil_token = 0.60
        else:
            output_cost_per_mil_token = 3.5
    
    elif "amazon.nova-lite-v1:0" in model_name:
        input_cost_per_mil_token = 0.00006 * 1000  # Convert to per million tokens
        output_cost_per_mil_token = 0.00024 * 1000  # Convert to per million tokens
    
    elif "amazon.nova-pro-v1:0" in model_name:
        input_cost_per_mil_token = 0.0008 * 1000
        output_cost_per_mil_token = 0.0032 * 1000

    else:
        raise ValueError(f"Unsupported model name: {model_name}")

    input_cost = input_tokens * input_cost_per_mil_token * 1e-6
    output_cost = output_tokens * output_cost_per_mil_token * 1e-6
    think_cost = think_tokens * output_cost_per_mil_token * 1e-6

    return input_cost, output_cost, think_cost


class SentimentEnum(str, Enum):
    POSITIVE = 'ポジティブ'
    NEUTRAL = '中立'
    NEGATIVE = 'ネガティブ'

class CategoryEnum(str, Enum):
    LECTURE_CONTENT = '講義内容'
    LECTURE_MATERIAL = '講義資料'
    OPERATION = '運営'
    OTHER = 'その他'

class ImportanceEnum(str, Enum):
    HIGH = '高'
    MEDIUM = '中'
    LOW = '低'

class CommonalityEnum(str, Enum):
    HIGH = '高'
    MEDIUM = '中'
    LOW = '低'


class EvalOutput(BaseModel):
    sentiment: SentimentEnum = Field(description="コメントに対する感情の分類")
    category: CategoryEnum = Field(description="コメントに対するカテゴリの分類")
    importance: ImportanceEnum = Field(description="コメントに対する重要度の分類")
    commonality: CommonalityEnum = Field(description="コメントに対する共通性の分類")

In [ ]:
data_days = ["Day1", "Day2", "Day3"]
models = ["amazon.nova-lite-v1:0", "amazon.nova-pro-v1:0"]
data_dir = os.path.join("..", "data")

for day in data_days:
    for model in models:
        print(f"Processing {day} with model {model}...")
        comment_df = pd.read_excel(os.path.join(data_dir, f"{day}_アンケート.xlsx"))
        
        total_score_column = "本日の総合的な満足度を５段階で教えてください。 "
        recommend_score_column = "親しいご友人にこの講義の受講をお薦めしますか？"
        comments_column = [col for col in comment_df.columns if "必須" in col or "任意" in col]
        # pandasからコメントを抽出して、辞書型の文字列のリストに変換
        total_scores = comment_df[total_score_column].tolist()
        recommend_scores = comment_df[recommend_score_column].tolist()
        comments = comment_df[comments_column].apply(
            lambda row: json.dumps(row.dropna().to_dict(), ensure_ascii=False), axis=1
        ).tolist()

        tasks = [
            generate_content_with_agent(
                model_id=model,
                query=[comment],
                output_type=EvalOutput,
                retries=3,
                temperature=0.2,
                max_tokens=2000,
                timeout=60,
            )
            for comment in comments
        ]

        semaphore = asyncio.Semaphore(50)
        async with semaphore:
            results = await asyncio.gather(*tasks, return_exceptions=True)

        # Process results
        processed_results = []
        for result in results:
            if isinstance(result, Exception):
                print(f"Error: {result}")
                result_dict = {
                    "sentiment": None,
                    "category": None,
                    "importance": None,
                    "commonality": None,
                    "total_cost": 0,
                    "is_error": True,
                }
            else:
                output, input_tokens, output_tokens, think_tokens = result

                input_cost, output_cost, think_cost = calculate_cost(
                    model_name=model,
                    input_tokens=input_tokens,
                    output_tokens=output_tokens,
                    think_tokens=think_tokens,
                )
                total_cost = input_cost + output_cost + think_cost

                result_dict = output.model_dump()
                for k, v in result_dict.items():
                        result_dict[k] = v.value
                result_dict.update({
                    "total_cost": total_cost,
                    "is_error": False,
                })

            processed_results.append(result_dict)

        # evaluation
        results_df = pd.DataFrame(processed_results)

        # is_errorのTrueを数えて、results_dfから削除
        error_count = results_df['is_error'].sum()
        print(f"Error Rate: {error_count / len(results_df):.2%}")
        comment_df = comment_df[~results_df['is_error']]
        results_df = results_df[~results_df['is_error']]

        # 感情のaccuracyを計算
        sentiment_pred = results_df['sentiment'].map({
            SentimentEnum.POSITIVE: 1,
            SentimentEnum.NEUTRAL: 0,
            SentimentEnum.NEGATIVE: -1
        })

        sentiment_act = comment_df[total_score_column].map({
            5: 1,
            4: 1,
            3: 0,
            2: -1,
            1: -1,
        })
        sentiment_accuracy = (sentiment_pred == sentiment_act).mean()
        print(f"Sentiment Accuracy: {sentiment_accuracy:.2%}")

        # 感情、重要度、共通性の掛け算からスコアを算出、点数との相関を計算
        importance_map = {
            ImportanceEnum.HIGH: 3,
            ImportanceEnum.MEDIUM: 2,
            ImportanceEnum.LOW: 1
        }
        commonality_map = {
            CommonalityEnum.HIGH: 3,
            CommonalityEnum.MEDIUM: 2,
            CommonalityEnum.LOW: 1
        }
        results_df['score'] = sentiment_pred * results_df['importance'].map(importance_map) * results_df['commonality'].map(commonality_map)
        # 1-5にスケールを合わせる
        results_df['score'] = (results_df['score'] + 9) / 18 * 4 + 1

        comment_df["点数"] = comment_df[total_score_column] * 0.5 + comment_df[recommend_score_column] * 0.5 / 2

        score_corr = results_df['score'].corr(comment_df['点数'])
        print(f"Score Correlation: {score_corr:.2f}")

        # コストを表示
        total_cost = results_df['total_cost'].sum()
        print(f"Total Cost: ${total_cost:.4f}")

        print("-" * 20)


Processing Day1 with model amazon.nova-lite-v1:0...
